# OWL2Vec* for Entity Linking

## Running OWL2Vec
### PART 2

In [4]:
# All the code here is from labs 8 & 9 from elective module taken (https://github.com/city-knowledge-graphs/python-2022)
# Codes for the Semantic Web Technologies and Knowledge Graphs module
# some code from gensim official github https://github.com/RaRe-Technologies/gensim
import pandas as pd

In [2]:
from owl2vec_star import owl2vec_star
from gensim.models import KeyedVectors

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1108)>


In [3]:
#Parameters:
# ontology_file
# config_file
# uri_doc
# lit_doc
# mix_doc
gensim_model = owl2vec_star.extract_owl2vec_model("./case_studies/snomedct/snomedct-201907_module_size_34467.owl", "./default.cfg", True, True, True)

output_folder="./cache/output/"

#Gensim format
gensim_model.save(output_folder+"SMDontology2.embeddings")
#Txt format
gensim_model.wv.save_word2vec_format(output_folder+"SMDontology2.embeddings.txt", binary=False)

INFO: Access the ontology ...
INFO: There are 817089 triples in the ontology
INFO: Calculate the ontology projection ...
INFO: Creating ontology graph projection...
INFO: 	Extracting subsumption triples
INFO: 		Time extracting subsumption: 3.5363709926605225 seconds 
INFO: 	Extracting equivalence triples
INFO: 		Time extracting equivalences: 6.971090793609619 seconds 
INFO: 	Extracting class membership triples.
INFO: 		Time extracting class membership: 121.70791220664978 seconds 
INFO: 	Extracting sameAs triples
INFO: 		Time extracting sameAs: 0.04391813278198242 seconds 
INFO: 	Extracting triples associated to 116676008
INFO: 		Time extracting triples for property: 1.6359660625457764 seconds 
INFO: 	Extracting triples associated to 116686009
INFO: 		Time extracting triples for property: 0.23434185981750488 seconds 
INFO: 	Extracting triples associated to 118169006
INFO: 		Time extracting triples for property: 0.2678563594818115 seconds 
INFO: 	Extracting triples associated to 11817100

## Loading Dataset

In [5]:
#Generar vectores de las entidades 
#import libraries
from gensim.models import Word2Vec
import pandas as pd

#import dataset
cometa = pd.read_csv("cometa.csv")
cometa = cometa.drop(columns=['Unnamed: 0'])
short_cometa = cometa.drop_duplicates()
cometa_entities = cometa['Term'].unique()
cometa_entities= pd.DataFrame(cometa_entities)
#entities dataset
cometa_entities = cometa_entities.rename(columns={0 : 'entites'}) #there are 6404 unique entities
cometa_entities.head(5)
short_cometa.head(5)

#accuracy dataset
#cometa_accuracy = cometa[['Term', 'SNOMED_Label']]
#cometa_accuracy = cometa_accuracy.rename(columns={0 : 'Entity'})

,Term,SNOMED_Label,SNOMED_ID
0,5FU,Fluorouracil,387172005
3,5HTP,Oxitriptan,73916008
6,6MP,Mercaptopurine,386835005
9,A1C,Ordinal level of hemoglobin A1c,443911005
12,A1C tests,Hemoglobin A1c measurement,43396009


In [7]:
wv = gensim_model.wv
wv.similarity('Fluorouracil', 'http://snomed.info/id/387172005')
wv.similarity('Oxitriptan', 'http://snomed.info/id/73916008')

0.74099284

In [8]:
cometa_entities

,entites
0,5FU
1,5HTP
2,6MP
3,A1C
4,A1C tests
...,...
6399,zolmitriptan
6400,zolpidem
6401,zonisamide
6402,zopiclone


## Loading embeddings and getting similarities

In [9]:
#Loading embedings 
from gensim.models import KeyedVectors
model = KeyedVectors.load("./cache/output/SMDontology2.embeddings", mmap='r')
wv = model.wv

INFO: loading KeyedVectors object from ./cache/output/SMDontology2.embeddings
INFO: loading wv recursively from ./cache/output/SMDontology2.embeddings.wv.* with mmap=r
INFO: loading vectors from ./cache/output/SMDontology2.embeddings.wv.vectors.npy with mmap=r
INFO: loading syn1neg from ./cache/output/SMDontology2.embeddings.syn1neg.npy with mmap=r
INFO: setting ignored attribute cum_table to None
INFO: Word2Vec lifecycle event {'fname': './cache/output/SMDontology2.embeddings', 'datetime': '2022-12-21T10:19:06.456568', 'gensim': '4.2.0', 'python': '3.8.1 (v3.8.1:1b293b6006, Dec 18 2019, 14:08:53) \n[Clang 6.0 (clang-600.0.57)]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'loaded'}


### Finf the top 10 ontology entities and check if any data is missing

In [10]:
#find the top 10 ontology entities linked to 
# sotre them in a dataset / list 
#AQUI!!!

d = []
h = []

cometa_entities2 = cometa_entities[0:20]

for i in cometa_entities['entites']:

    while True:
        try: 
            print(i)
            similarity = wv.most_similar(positive=[i], topn=10)
            print(similarity)
            d.append(
                {
                    'Entity': i,
                    'Conceps': similarity 
                }
                )
            break
            
        except KeyError:
            print("not found " + i)
            h.append(i)
            break

# retrieved in datasets the concepts that are present and the ones that are not 
print("Len d ", len(d))
print("Len h ", len(h)) 

#align d with cometa original to have "entity" vs "uri"
entity_data = pd.DataFrame(d)
missing_entity_data = pd.DataFrame(h)
sim_dataset = pd.merge(entity_data, short_cometa, how='inner', left_on = 'Entity', right_on = 'Term')


5FU
not found 5FU
5HTP
not found 5HTP
6MP
[('Removal of thrombus of lower limb vein', 0.9824106097221375), ('Fluid sample bicarbonate measurement (procedure)', 0.9815549254417419), ('ANF', 0.9802906513214111), ('Immunoglobulin A level', 0.9802858829498291), ('Urine glutamine level', 0.980201244354248), ('Plasma lipid measurement (procedure)', 0.9797996282577515), ('Rifampin measurement', 0.9797406792640686), ('Ultrasound of right knee region', 0.9796938300132751), ('Chronic iridocyclitis of right eye', 0.9796906113624573), ('Estrogen measurement, urine (procedure)', 0.9796429872512817)]
A1C
not found A1C
A1C tests
not found A1C tests
A1Cs
not found A1Cs
ABVD
not found ABVD
ABVD chemo
not found ABVD chemo
AC1
not found AC1
ACE inhibitor
[('Magnetic resonance imaging of both wrists with contrast', 0.9845643639564514), ('Substance with beta adrenergic receptor agonist mechanism of action', 0.9838247299194336), ('Open wound of hip', 0.9834717512130737), ('Substance with antifungal mechanis

In [12]:
sim_dataset.head()

,Entity,Conceps,Term,SNOMED_Label,SNOMED_ID
0,6MP,"[(Removal of thrombus of lower limb vein, 0.98...",6MP,Mercaptopurine,386835005
1,ACE inhibitor,[(Magnetic resonance imaging of both wrists wi...,ACE inhibitor,Angiotensin-converting enzyme inhibitor,734579009
2,AIDS,"[(Ranitidine (substance), 0.9795656204223633),...",AIDS,Acquired immune deficiency syndrome,62479008
3,ANA,"[(Atrophic vulvitis, 0.9846677780151367), (Con...",ANA,Antinuclear antibody,45530001
4,Abscess,"[(Periodontal abscess, 0.9498739242553711), (L...",Abscess,Abscess,128477000


In [14]:
#sim_dataset = pd.merge(entity_data, short_cometa, how='inner', left_on = 'Entity', right_on = 'Term')
from nltk.tokenize import word_tokenize
term_unique = sim_dataset['Term'].values.tolist()
term_unique = list(map(lambda x: x.lower(), term_unique))
term_tokenized = [word_tokenize(i) for i in term_unique]
len(term_tokenized) #2848

2848

In [17]:
# PREPOCESING OF SNOMED_ID -- URI
#add a prefix to the snomed concepts so we can see the URI -- http://snomed.info/id/
append_str = 'http://snomed.info/id/'

#similarity row:
snomedid_unique = sim_dataset['SNOMED_ID'].map(str)
#print(len(term_unique), len(snomedid_unique))

# Append suffix / prefix to strings in list
snomedid_unique_str = [append_str + sub for sub in snomedid_unique]
  
# Printing result
#print("list after prefix addition : " + str(snomedid_unique_str))
len(snomedid_unique_str)  #2848

2848

In [19]:
#ESTE 
# with n_similarity function from gensim:

sim_dataset = term_tokenized
list_of_words = snomedid_unique_str
df_2 = []
for i in range(len(sim_dataset)):
    #print(sim_dataset_5[i], list_of_words_5[i])
    #print(wv.n_similarity(sim_dataset_5[i], list_of_words_5[i]))
    #print(wv.cosine_similarities(list_of_words[i], array_uri[i]))
    similarity = wv.n_similarity(sim_dataset[i], list_of_words[i])
    df_2.append({

        'Reddit Entity' : sim_dataset[i],
        'SNOMED Link' : list_of_words[i],
        'Similarity' : similarity,
        
    })

df_2 = pd.DataFrame(df_1)

In [21]:
df_2.describe()

,Similarity
count,2848.000000
mean,0.449207
std,0.086059
min,0.205977
25%,0.389978
50%,0.449756
75%,0.509171
max,0.801247


In [27]:
#with similarity function
sim_dataset_5 = term_tokenized
list_of_words_5 = snomedid_unique_str
data2 = []

for i in range(len(sim_dataset_5)-1):
    #print(sim_dataset_5[i], list_of_words_5[i])
    #print(wv.similarity(sim_dataset_5[i], list_of_words_5[i]))
    similarity222 = wv.similarity(sim_dataset_5[i], list_of_words_5[i])
    #print(wv.cosine_similarities(list_of_words[i], array_uri[i]))
    data2.append({

        'Reddit Entity' : sim_dataset_5[i],
        'SNOMED Link' : list_of_words_5[i],
        'Similarity' : similarity222,
        
    })

data2 = pd.DataFrame(data2)

In [28]:
data2.describe()

,Reddit Entity,SNOMED Link,Similarity
count,2847,2847,2847
unique,2698,2321,2716
top,[pityriasis],http://snomed.info/id/55300003,[0.7673948]
freq,3,5,3


In [31]:
df_2.loc[df_2['Similarity'] == 0.0] #1573 row were not able to find similarity

,Reddit Entity,SNOMED Link,Similarity
